In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
#reload modules
%load_ext autoreload
%autoreload 2

In [ ]:
import join_data as jd

In [ ]:
from helpers import *

In [ ]:
filesavelabel = 'publicdatacategory_'
predicted_delays_filename = './data_est_report_delays/category_borough_interaction_model_delays.csv'

# filesavelabel = 'publicdatarisk_'
# predicted_delays_filename = './data_est_report_delays/risk_borough_interaction_model_delays.csv'

# Load and join the public data

In [ ]:
mergeddfpublic = jd.pipeline(directory = '../data/', FSfilename = 'FSR_221022.csv', FIfilename = 'FI_221022.csv', FWOfilename = 'FWO_221022.csv', FRAfilename = 'FRA_221024.csv')

In [ ]:
mergeddfpublic.columns

### Load + join model estimates for reporting delays

We have a dataframe where we have predicted report delays for many incidents (that were used to train the model). We also want to have predicted reporting delays for other incidents. So what we will do is extract coefficients from that dataframe (we can do this perfectly except for Bayesian noise) and apply the model to all the incidents.

In [ ]:
predicted_report_delays = pd.read_csv(predicted_delays_filename)
predicted_report_delays = predicted_report_delays.rename(columns = {'SRID': 'OBJECTID', 'delay': 'reporting_delay'})

In [ ]:
predicted_report_delays.head()

In [ ]:
predicted_report_delays_joinedwithfeatures = pd.merge(predicted_report_delays, mergeddfpublic, on='OBJECTID', how='left', suffixes=('', '_public')).drop_duplicates(subset = 'OBJECTID')

In [ ]:
predicted_report_delays.count()

In [ ]:
predicted_report_delays_joinedwithfeatures.count()

In [ ]:
#statsmodels from formula import
import statsmodels.formula.api as smf

In [ ]:
predicted_report_delays_joinedwithfeatures.columns

In [ ]:
cols = ['SRCategory', 'Borough', 'RiskRating', 'InspectionTPCondition', 'TreePointDBH']

predicted_report_delays_joinedwithfeatures.loc[:, 'reporting_rate'] = predicted_report_delays_joinedwithfeatures.eval('1/reporting_delay')

predicted_report_delays_joinedwithfeatures.loc[:, 'log_reporting_rate'] = predicted_report_delays_joinedwithfeatures.eval('log(1/reporting_delay)')


#regress reporting delay on category, borough, and risk
model = smf.ols(formula='log_reporting_rate ~ 1 + C(SRCategory) + C(Borough) + C(Risk_coded) + C(Borough):C(SRCategory) + InspectionTPCondition + np.log(TreePointDBH + 1)', data=predicted_report_delays_joinedwithfeatures).fit()
model.summary()

In [ ]:
#now apply the model to nodups_rightdate
predicted_report_delays_joinedwithfeatures['log_reporting_rate_predicted'] = model.predict(predicted_report_delays_joinedwithfeatures)
predicted_report_delays_joinedwithfeatures['reporting_delay_predicted'] = predicted_report_delays_joinedwithfeatures.eval('1/exp(log_reporting_rate_predicted)')

In [ ]:
sns.jointplot(x='reporting_delay_predicted', y='reporting_delay', data=predicted_report_delays_joinedwithfeatures.query('reporting_delay_predicted < 100 and reporting_delay < 100'), kind='hex')
plt.xlim((0, 100))
plt.ylim((0, 100))

Ok, we have a good model of reporting delays, we can apply it to all the public data now

# Delay analysis final data preparation

First, combine multiple reports of the same incident so that I have one row per unique incident. Take first inspection and report date as the right date.

In [ ]:
wofinishdatecolumn = 'ActualFinishDate' # 'WOClosedDate'

In [ ]:
nodups = mergeddfpublic[['IncidentGlobalID','SRCategory','SRCreatedDate', 'InspectionDate',wofinishdatecolumn,'Risk_coded','RiskRating','Borough','InspectionTPCondition', 'InspectionTPStructure',
       'TreePointDBH']].groupby('IncidentGlobalID').agg(
    {'SRCreatedDate': 'min', 'InspectionDate': 'min', wofinishdatecolumn: 'min', 'SRCategory': 'first', 'Risk_coded': 'first', 'Borough': 'first', 'RiskRating': 'first', 'InspectionTPCondition': 'first', 'InspectionTPStructure': 'first',
       'TreePointDBH': 'first'}).reset_index()
nodups['inspection_delay'] = (nodups['InspectionDate'] - nodups['SRCreatedDate']).dt.total_seconds() / 3600 /24
nodups['work_delay'] = (nodups[wofinishdatecolumn] - nodups['InspectionDate']).dt.total_seconds() / 3600 /24
nodups[['SRCreatedDate','InspectionDate', wofinishdatecolumn]].describe(datetime_is_numeric=True)

In [ ]:
nodups_rightdate = nodups[(nodups['SRCreatedDate'] >= '2017-06-30') & (nodups['SRCreatedDate'] < '2020-07-01')]

In [ ]:
nodups_rightdate = nodups_rightdate[nodups_rightdate.SRCategory.isin(predicted_report_delays_joinedwithfeatures.SRCategory.unique())]

In [ ]:
nodups_rightdate[['SRCreatedDate','InspectionDate', 'work_delay']].describe(datetime_is_numeric=True)

In [ ]:
nodups_rightdate.loc[:,'log_reporting_rate'] = model.predict(nodups_rightdate)
nodups_rightdate.loc[:,'reporting_delay'] = nodups_rightdate.eval('1/exp(log_reporting_rate)')

In [ ]:
nodups_rightdate.count()

In [ ]:
nodups_rightdate.groupby(['SRCategory', 'Borough'])['reporting_delay', 'inspection_delay', 'work_delay'].median()

In [ ]:
#only looking at the inspected set
nodups_rightdate = nodups_rightdate.dropna(subset = ['reporting_delay'])

In [ ]:
addressed = nodups_rightdate.groupby(['SRCategory', 'Borough'])['inspection_delay', 'work_delay'].agg(lambda x: 1-np.mean(np.isnan(x))).reset_index()
addressed

In [ ]:
addressed = nodups_rightdate.groupby(['Risk_coded', 'Borough'])['inspection_delay', 'work_delay'].agg(lambda x: 1-np.mean(np.isnan(x))).reset_index()
addressed

In [ ]:
# #turn inpsection and work delays into rows instead of columns
# addressed = addressed.melt(id_vars=['SRCategory', 'Borough'], value_vars=['inspection_delay', 'work_delay'], var_name='delay_type', value_name='percent_addressed')

In [ ]:
# addressed = addressed.rename({'percent_addressed': 'Fraction addressed', 'delay_type': 'Action'}, axis=1)
# addressed.loc[:, 'Action'] = addressed.loc[:, 'Action'].str.replace('inspection_delay', 'Inspection').str.replace('work_delay', 'Work order')
# plot = sns.catplot(
#     data=addressed.query('SRCategory == "Hazard" and Action=="Work order"'), kind="bar",
#     x="Borough", y="Fraction addressed", hue="Action",
#     errorbar="sd", palette= ['green'], legend_out = False, #, alpha=.6, height=6
#     order = ['Manhattan', 'Queens', 'Staten Island', 'Bronx', 'Brooklyn']
# )
# legend = plot._legend
# legend.set_frame_on(False)
# plt.ylim(0, 1.1)
# plt.xlabel(None)
# plt.savefig(f'plots/{filesavelabel}hazard_fractionaddressed.pdf', bbox_inches='tight')

# Plotting

<!-- ## What fraction of incidents are actually addressed -->

## Delays conditional on addressed

In [ ]:
# plot_bar_by_type(nodups_rightdate, typecol = 'SRCategory', othergroupby = 'Borough', impute_missing_work_order = False, label = f'{filesavelabel}')

In [ ]:
nodups_rightdate = nodups_rightdate.sort_values(by = 'Risk_coded', ascending = True)

In [ ]:
import helpers

In [ ]:
helpers.plot_bar_by_type(nodups_rightdate.dropna(subset = ['inspection_delay']), typecol = 'SRCategory', othergroupby = 'Borough', impute_missing_work_order = False, label = f'{filesavelabel}category_notimputed_')

In [ ]:
helpers.plot_bar_by_type(nodups_rightdate.dropna(subset = ['inspection_delay']), typecol = 'SRCategory', othergroupby = 'Borough', impute_missing_work_order = True, label = f'{filesavelabel}category_imputed_')